## Reconcile

In [1]:
import pandas as pd
import numpy as np
import configparser
import datetime as dt
import re
from functools import reduce
# import matplotlib as mpl
# from matplotlib import pyplot as plt
# from matplotlib.gridspec import GridSpec

# mpl.rcParams['axes.spines.top'] = False
# mpl.rcParams['axes.spines.right'] = False

In [2]:
''' Config '''

config = configparser.ConfigParser()
config.read('../my_config.ini')
config.sections()

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
op_file_name = f'Agg_Perf_Nationwide-{str_curr_dt}'
op_file_name

# data_src = f"../data/raw/Agg_Perf_Nationwide-20240605.csv"

'Agg_Perf_Nationwide-20240606'

In [3]:
''' Rawdata '''

data_src = f"../data/raw/{op_file_name}.csv"
df = pd.read_csv(data_src, low_memory=False)

cols = ['TM_KEY_DAY', 'TM_KEY_WK', 'TM_KEY_MTH', 'TM_KEY_QTR', 'TM_KEY_YR', 'CENTER', 'METRIC_GRP', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME', 
        'ACTUAL_AS_OF', 'AGG_TYPE', 'RR_IND', 'GRY_IND', 'UOM', 'PERIOD', 'ACTUAL_SNAP', 'ACTUAL_AGG', 'TARGET_SNAP', 'TARGET_AGG', 'BASELINE_SNAP', 'BASELINE_AGG', 'PPN_TM']

df = df[cols]

''' Filters '''
df = df[df['TM_KEY_YR'] == 2024] # YEAR filter
# df = df[df['TM_KEY_MTH'] >= 202401] # MONTH filter

df.tail()

# ''' Fix Metric CD list '''

# ''' Fix Agg Column Name(Total Revenue) '''
# bg_col_list = ['POST_DTAC', 'POST_TMH', 'PRE_DTAC', 'PRE_TMH', 'TOL', 'TVS']

,TM_KEY_DAY,TM_KEY_WK,TM_KEY_MTH,TM_KEY_QTR,TM_KEY_YR,CENTER,METRIC_GRP,PRODUCT_GRP,COMP_CD,METRIC_CD,...,GRY_IND,UOM,PERIOD,ACTUAL_SNAP,ACTUAL_AGG,TARGET_SNAP,TARGET_AGG,BASELINE_SNAP,BASELINE_AGG,PPN_TM
555505,20240605,2024023,202406,20242,2024,Revenue,Accounts,TOL,TRUE,TB3S020604A,...,GYR,accounts,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-06 10:34:07
555506,20240605,2024023,202406,20242,2024,Revenue,Sales,Prepaid,ALL,B1R001000AI,...,GYR,baht,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-06 10:34:07
555507,20240605,2024023,202406,20242,2024,Revenue,Subs,Postpaid,TRUE,TB2S000701AF,...,RYG,%,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-06 10:34:07
555508,20240605,2024023,202406,20242,2024,Revenue,Sales,Postpaid,ALL,B2S000100AE,...,GYR,subs,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-06 10:34:07
555509,20240605,2024023,202406,20242,2024,Revenue,Subs,Postpaid,ALL,B2S020300,...,GYR,subs,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-06 10:34:07


In [4]:
print(f'\ndf : {df.shape[0]} rows, {df.shape[1]} columns')


df : 217257 rows, 24 columns


In [7]:
# df_tmp = df[['METRIC_GRP', 'METRIC_CD', 'METRIC_NAME']].drop_duplicates().sort_values(['METRIC_GRP', 'METRIC_CD'])
# df_tmp

df[['METRIC_GRP']].drop_duplicates().sort_values(['METRIC_GRP']).reset_index(drop=True)

,METRIC_GRP
0,Accounts
1,Digital Growth Champion
2,Market Share
3,Retention & Churn
4,Revenue
5,Sales
6,Subs
7,NaN


In [8]:
''' Function '''

def view_group(grp, cd, name):
    grp = grp
    cd = cd
    name = name
    flag = ''

    # CORP & MCOM
    if re.search('C$|H$|MCOM$', cd) and (not re.search('A[A-K]$', cd)): flag = 'CORP & MCOM'
    elif re.search('CUS$', cd): flag = 'Cust Location'
    # Revenue
    elif grp == 'Revenue' and any(x in name for x in ['New Revenue', 'Existing Revenue']): flag = 'New/Existing'
    elif grp == 'Revenue' and any(x in name for x in ['Paid Amount', 'On Due', 'Overdue']): flag = 'Paid Amount'
    elif grp == 'Revenue' and any(x in name for x in ['Revenue']): flag = 'Revenue'
    # Sales
    elif grp == 'Sales' and any(x in name for x in ['Inflow M1']): flag = 'Inflow M1'
    elif grp == 'Sales' and any(x in name for x in ['Inflow M2']): flag = 'Inflow M2'
    elif grp == 'Sales' and any(x in name for x in ['Gross Add']): flag = 'Gross Adds'
    elif grp == 'Sales' and any(x in name for x in ['%AP']): flag = '%AP'
    elif grp == 'Sales' and any(x in name for x in ['AP 1D']): flag = 'AP 1D'
    elif grp == 'Sales' and any(x in name for x in ['AP In Month']): flag = 'AP MTH'
    elif grp == 'Sales' and any(x in name for x in ['Activation Subs']): flag = 'Activation'
    elif grp == 'Sales' and any(x in name for x in ['Conversion']): flag = '%Conversion'
    elif grp == 'Sales' and any(x in name for x in ['GA ARPU', 'GA RC']): flag = 'GA ARPU/RC'
    # Subs
    elif grp == 'Subs' and any(x in name for x in ['Net Add']): flag = 'Net Adds'
    elif grp == 'Subs' and any(x in name for x in ['%NAD']): flag = '%NAD'
    elif grp == 'Subs' and any(x in name for x in ['%M4']): flag = '%M4'
    elif grp == 'Subs' and any(x in name for x in ['Reported Sub']): flag = 'Reported Subs'
    elif grp == 'Subs' and any(x in name for x in ['Usage Subs', 'Active Caller', 'Active Subs']): flag = 'Active Subs'
    elif grp == 'Subs' and any(x in name for x in ['NAD']): flag = 'NAD'
    elif grp == 'Subs' and any(x in name for x in ['Revenue Subs']): flag = 'Rev Subs'
    # MKS
    elif grp == 'Market Share' and any(x in name for x in ['Broadband']): flag = '%BB MKS'
    elif grp == 'Market Share' and (not any(x in name for x in ['Broadband'])) & any(x in name for x in ['(Subs)']): flag = 'MB MKS(Subs)'
    elif grp == 'Market Share' and (not any(x in name for x in ['Broadband', '(Subs)'])): flag = '%MB MKS'
    # Churn
    elif grp == 'Retention & Churn' and any(x in name for x in ['Churn Subs']): flag = 'Churn Subs'
    elif grp == 'Retention & Churn' and any(x in name for x in ['Churn Rate']): flag = '%Churn Rate'
    # Others
    elif any(x in name for x in ['ARPU']): flag = 'ARPU'
    elif any(x in name for x in ['SubBase']): flag = 'SubBase'
    elif any(x in name for x in ['New Subs']): flag = 'New Subs'
    elif any(x in name for x in ['Silent']): flag = 'Silent'
    elif any(x in name for x in ['60DPD']): flag = '60DPD'
    elif any(x in name for x in ['Quality']): flag = 'Quality'
    else: flag = 'Unknown'
 
    return flag

In [10]:
# df_tmp['FLAG'] = df_tmp['METRIC_GRP'].apply(lambda x: group_flag(str(x)))
df['VIEW_GRP'] = df.apply(lambda x: view_group(str(x['METRIC_GRP']), str(x['METRIC_CD']), str(x['METRIC_NAME'])), axis=1)

# df.sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD'])

# df[df['METRIC_GRP']=='Revenue'].sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD'])
# df[df['METRIC_GRP']=='Market Share'].sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD'])
# df[df['METRIC_GRP']=='Retention & Churn'].sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD'])

# df[df['VIEW_GRP']=='Revenue'].sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD'])
# df[df['VIEW_GRP']=='Unknown'].sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD'])
# df[df['VIEW_GRP']!='CORP & MCOM'].sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD'])

df[~df['VIEW_GRP'].isin(['CORP & MCOM', 'Cust Location', 'Unknown'])].sort_values(['VIEW_GRP', 'METRIC_GRP', 'METRIC_CD']).reset_index(drop=True)

# df[['VIEW_GRP']].drop_duplicates().sort_values(['VIEW_GRP'])
# df[['METRIC_GRP', 'VIEW_GRP']].drop_duplicates().sort_values(['METRIC_GRP', 'VIEW_GRP'])

,TM_KEY_DAY,TM_KEY_WK,TM_KEY_MTH,TM_KEY_QTR,TM_KEY_YR,CENTER,METRIC_GRP,PRODUCT_GRP,COMP_CD,METRIC_CD,...,UOM,PERIOD,ACTUAL_SNAP,ACTUAL_AGG,TARGET_SNAP,TARGET_AGG,BASELINE_SNAP,BASELINE_AGG,PPN_TM,VIEW_GRP
0,20240101,2024001,202401,20241,2024,Revenue,Sales,Prepaid,ALL,B1S001000,...,%,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-06 10:34:07,%AP
1,20240102,2024001,202401,20241,2024,Revenue,Sales,Prepaid,ALL,B1S001000,...,%,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-06 10:34:07,%AP
2,20240103,2024001,202401,20241,2024,Revenue,Sales,Prepaid,ALL,B1S001000,...,%,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-06 10:34:07,%AP
3,20240104,2024001,202401,20241,2024,Revenue,Sales,Prepaid,ALL,B1S001000,...,%,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-06 10:34:07,%AP
4,20240105,2024001,202401,20241,2024,Revenue,Sales,Prepaid,ALL,B1S001000,...,%,M,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-06 10:34:07,%AP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176718,20240601,2024022,202406,20242,2024,Revenue,Subs,TOL,TRUE,TB3S020604,...,subs,M,3210415.0,3210415.0,NaN,NaN,NaN,NaN,2024-06-06 10:34:07,SubBase
176719,20240602,2024022,202406,20242,2024,Revenue,Subs,TOL,TRUE,TB3S020604,...,subs,M,3211196.0,3211196.0,NaN,NaN,NaN,NaN,2024-06-06 10:34:07,SubBase
176720,20240603,2024023,202406,20242,2024,Revenue,Subs,TOL,TRUE,TB3S020604,...,subs,M,3212049.0,3212049.0,NaN,NaN,NaN,NaN,2024-06-06 10:34:07,SubBase
176721,20240604,2024023,202406,20242,2024,Revenue,Subs,TOL,TRUE,TB3S020604,...,subs,M,NaN,3212049.0,NaN,NaN,NaN,NaN,2024-06-06 10:34:07,SubBase
